# Quality Check Annotation Bounding Boxes.
Annotations are provided as point (x,y) coordinates and the bounding box on the NFT object is estimated using watershed. Use the interactive in this notebook to view bounding boxes and, when needed, adjust them. 

In [ ]:
# Interactive
from os.path import join
from pandas import read_csv
import cv2 as cv
import matplotlib.pyplot as plt
from ipywidgets import (
    IntRangeSlider, Layout, fixed, interact, Button, IntProgress, interactive, 
    VBox, HBox
)
from IPython.display import display

from nft_helpers.utils import load_yaml, imread
from nft_helpers.box_and_contours import line_to_xys

IMG_SIZE = 500  # Size of images, hard-coded from running 1.download-annotations.py
COLORS = {'iNFT': (255, 0, 0), 'Pre-NFT': (0, 0, 255)}
DESC_STYLES = {'description_width': 'initial', 'font_weight': 'bold'}
cf = load_yaml()

# interactive to qc bounding boxes on point annotations NFTs
csv_fp = join(cf.codedir, 'csvs/annotations.csv')  # read the annotations dataframe
df = read_csv(csv_fp)

# sort
df = df.sort_values(by=['status', 'url_to_parent_roi', 'pt_x', 'pt_y']).reset_index(drop=True)
M = len(df)

start_idx = df[df.status == 'needs checking']

# create some global vars
x1, x2, y1, y2 = None, None, None, None


def show_annotation(img, color, y1y2, x1x2, url):
    global x1, x2, y1, y2
    # draw the coordinate
    x1, x2 = x1x2
    y1, y2 = y1y2

    h, w = img.shape[:2]
    img = cv.rectangle(img.copy(), (x1, y1), (x2, y2), color, 2)
    img = cv.circle(img, (int(w / 2), int(h / 2)), 7, color, 2)
    
    plt.figure(figsize=(6,6))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    print(url)


def select_r(i):
    r = df.iloc[i-1]
    img = imread(r.im_path)

    coords = line_to_xys(r.box_coords, shift=(r.im_left, r.im_top))

    x1, y1 = coords[0]
    x2, y2 = coords[1]

    x_slider = IntRangeSlider(min=0, max=IMG_SIZE, continuous_update=False, value=[x1, x2], description='x coords:', layout=Layout(width='500px'))
    y_slider = IntRangeSlider(min=0, max=IMG_SIZE, continuous_update=False, value=[y1, y2], description='y coords:', layout=Layout(width='500px'))
    _ = interact(show_annotation, img=fixed(img), color=fixed(COLORS[r.label]), x1x2=x_slider, y1y2=y_slider, url=fixed(r.url_to_im))


def confirm_fnc(_):
    global df, idx_select
    # get the index of the slider
    current_i = idx_select.value
    current_r = df.iloc[current_i - 1]

    # update the box coordinates and save
    box_coords = f'{x1+current_r.im_left} {y1+current_r.im_top} {x2+current_r.im_left} {y2+current_r.im_top}'

    df.loc[current_i - 1, 'status'] = 'checked'

    if box_coords != current_r.box_coords:
        df.loc[current_i - 1, 'box_coords'] = box_coords

    next_i = current_i + 1

    # tick to next image if not at the last image
    if next_i <= M:
        idx_select.description = f'Progress: {next_i} of {M} ({next_i / M * 100:.3f}%)'
        idx_select.value = next_i
    else:
        print('All done!')


def undo_fnc(_):
    global df, idx_select

    # go back only if you can
    current_i = idx_select.value

    previous_i = current_i - 1

    if previous_i > 0:  # can't go to zero
        # get the previous r
        df.loc[previous_i - 1, 'status'] = 'needs checking'  # switch the previous status to needs checking

        idx_select.description = f'Progress: {previous_i} of {M} ({previous_i / M * 100:.3f}%)'
        idx_select.value = previous_i


def save_fnc(_):
    global df
    df.to_csv(csv_fp, index=False)


if len(start_idx):
    start_idx = start_idx.index[0] + 1

    # create widgets
    confirm_bn = Button(description='next', botton_style='success', layout=Layout(height='40px'))
    confirm_bn.style.button_color = 'green'
    undo_bn = Button(description='previous', layout=Layout(height='40px'))
    undo_bn.style.button_color = 'red'
    save_bn = Button(description='save csv', layout=Layout(height='40px'))
    save_bn.style.button_color = 'cyan'
    idx_select = IntProgress(value=start_idx, max=M, min=1, orientation='horizontal', style=DESC_STYLES, description=f'Progress: {start_idx} of {M} ({start_idx / M * 100:.3f}%)')

    # add functionality
    confirm_bn.on_click(confirm_fnc)
    undo_bn.on_click(undo_fnc)
    save_bn.on_click(save_fnc)

    w = interactive(select_r, i=idx_select)
    out = w.children[-1]

    # display
    canvas = VBox([
        HBox([save_bn, idx_select]),
        HBox([undo_bn, confirm_bn]),
        out
    ])
    display(canvas)
else:
    print('All annotation images have been checked - good to go!')